<a href="https://colab.research.google.com/github/raneymjohnGit/Austin_TX_House_Listings/blob/jayanbranch/Analysis/Testing_Data_Transformation_for_Front_End.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copy paste this code
# Install psycopg2-binary
! pip install psycopg2-binary 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.0 MB 11.2 MB/s 


In [ ]:
# Copy paste this code
# URL for Heroku DB
raw_db_url='dpaclsxjrpfluk:4fa14e6fdb846bd14d1a9eda261d554fab9688f2f4dd5483bc38d94cdee010ad@ec2-3-219-52-220.compute-1.amazonaws.com:5432/d7s0s0hs0a5lar'

In [ ]:
 # Copy paste this code
 # Import SQL Alchemy, format url string for SQLAlchemy
 
import subprocess
from sqlalchemy.engine.create import create_engine

# Convert binary string to a regular string & remove the newline character
db_url = raw_db_url

# Convert "postgres://<db_address>"  --> "postgresql+psycopg2://<db_address>" needed for SQLAlchemy
final_db_url = "postgresql+psycopg2://" + db_url.lstrip("postgres://")  # lstrip() is more suitable here than replace() function since we only want to replace postgres at the start!


# Create SQLAlchemy engine
# ------------------------
engine = create_engine(final_db_url)

# Read in cleaned data from SQL as DataFrame

In [ ]:
# SQL to dataframe using sqlalchemy
import pandas as pd

df = pd.read_sql_table('cleaned_data_1', engine)
df.head()

,index,zpid,city,streetAddress,zipcode,description,latitude,longitude,propertyTaxRate,garageSpaces,...,avgSchoolSize,MedianStudentsPerTeacher,numOfBathrooms,numOfBedrooms,numOfStories,month_year,latest_saledate_dt,dec_2019_price,jan_2021_price,price_per_sqft
0,6702,29415933,austin,5804 Bullard Dr,78757,"5804 Bullard Dr, Austin, TX 78757 is a single ...",30.336634,-97.750031,1.98,3,...,1153,16,2.0,3,1,2019-06,2019-06-21,470317,547114,202
1,4729,58304707,austin,823 Walnut Creek Dr,78753,"823 Walnut Creek Dr, Austin, TX 78753 is a sin...",30.386883,-97.668449,1.98,3,...,814,15,2.0,3,1,2019-02,2019-02-27,418299,486603,211
2,7501,29482307,austin,4410 Garnett St,78745,Complete remodel of a gorgeous vintage home. O...,30.218693,-97.770554,1.98,3,...,792,13,2.0,4,1,2019-07,2019-07-02,598560,696298,343
3,5146,29513449,austin,11412 Gun Fight Ln,78748,Large move-in ready Shady Hollow home loaded w...,30.162312,-97.854210,1.98,3,...,1633,17,4.0,5,2,2019-03,2019-03-20,481300,559890,137
4,5256,29479870,austin,1311 Casey St,78745,"1311 Casey St, Austin, TX 78745 is a single fa...",30.225712,-97.781837,1.98,3,...,803,12,2.0,3,1,2019-03,2019-03-11,374344,435470,300


In [ ]:
# Drop non-numeric columns that will not be used for models, erroneus columns like index and jan_2021_price, and columns that are calculated from variables like price_per_sqft

df = df.drop(columns=['zpid', 'index', 'city', 'streetAddress', 'description', 'latest_saledate', 'latest_salemonth', 'latest_saleyear', 'month_year', 'latest_saledate_dt', 'jan_2021_price', 'price_per_sqft', 'homeType', 'latestprice'])


In [ ]:
df['zipcode'] = df['zipcode'].astype(str)

In [ ]:
import pandas as pd # data processing
import numpy as np # working with arrays
import matplotlib.pyplot as plt # visualization
import seaborn as sb # visualization
from termcolor import colored as cl # text customization

from sklearn.model_selection import train_test_split # data split
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score as evs # evaluation metric
from sklearn.metrics import r2_score as r2 # evaluation metric
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df_front_end = df[['zipcode', 'yearBuilt', 'lotSizeSqFt', 'livingAreaSqFt', 'avgSchoolRating','dec_2019_price']]
df_front_end.shape

(8011, 6)

In [ ]:
df_front_end.head()

,zipcode,yearBuilt,lotSizeSqFt,livingAreaSqFt,avgSchoolRating,dec_2019_price
0,78757,1958,10018.0,2226.0,6.666667,470317
1,78753,1961,10018.0,1706.0,5.000000,418299
2,78745,1941,15246.0,1676.0,3.333333,598560
3,78748,1988,7361.0,3279.0,7.333333,481300
4,78745,1958,9931.0,1168.0,4.000000,374344


In [ ]:
X = df_front_end.drop(["dec_2019_price"],axis = 1)
y = df_front_end.dec_2019_price
ct = ColumnTransformer([("cat", OneHotEncoder(handle_unknown = "ignore"),[0])], remainder="passthrough")
rf_model = RandomForestRegressor()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)
model = Pipeline([("preprocessing",ct),("model",rf_model)]).fit(X_train,y_train)
print(model.named_steps["preprocessing"].get_feature_names())
Pipeline([("preprocessing",ct),("model",rf_model)]).score(X_test,y_test)



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name']. An error will be raised in 1.2.
  FutureWarning,


['cat__x0_78617', 'cat__x0_78660', 'cat__x0_78702', 'cat__x0_78703', 'cat__x0_78704', 'cat__x0_78705', 'cat__x0_78717', 'cat__x0_78721', 'cat__x0_78722', 'cat__x0_78723', 'cat__x0_78724', 'cat__x0_78725', 'cat__x0_78726', 'cat__x0_78727', 'cat__x0_78728', 'cat__x0_78729', 'cat__x0_78730', 'cat__x0_78731', 'cat__x0_78732', 'cat__x0_78733', 'cat__x0_78735', 'cat__x0_78736', 'cat__x0_78737', 'cat__x0_78739', 'cat__x0_78741', 'cat__x0_78744', 'cat__x0_78745', 'cat__x0_78746', 'cat__x0_78747', 'cat__x0_78748', 'cat__x0_78749', 'cat__x0_78750', 'cat__x0_78751', 'cat__x0_78752', 'cat__x0_78753', 'cat__x0_78754', 'cat__x0_78756', 'cat__x0_78757', 'cat__x0_78758', 'cat__x0_78759', 'x1', 'x2', 'x3', 'x4']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name']. An error will be raised in 1.2.
  FutureWarning,


0.7017517863175484

In [ ]:
test_input = [['78757',	1958,	10018.0,	2226.0,	6.666667]]

In [ ]:
test_input_transformed = ct.transform(test_input)
print(test_input_transformed.toarray())

[[0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00
  1.958000e+03 1.001800e+04 2.226000e+03 6.666667e+00]]


In [ ]:
rf_model.predict(test_input_transformed)

array([540152.27])

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# loading joblib dependency
import joblib

# save best_random model for front end, filename-model_jlib
joblib.dump(rf_model, '/content/drive/MyDrive/MachineLearningModels/rf_test-model_jlib')

['/content/drive/MyDrive/MachineLearningModels/rf_test-model_jlib']

In [ ]:
# loading joblib dependency
import joblib

# save best_random model for front end, filename-model_jlib
joblib.dump(model, '/content/drive/MyDrive/MachineLearningModels/rf_test1-model_jlib')

['/content/drive/MyDrive/MachineLearningModels/rf_test1-model_jlib']